In [107]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [108]:
data = pd.read_csv('data/abalone.csv')

In [109]:
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
data['Sex']

0       2
1       2
2       0
3       2
4       1
       ..
4172    0
4173    2
4174    2
4175    0
4176    2
Name: Sex, Length: 4177, dtype: int64

In [110]:
X = data.drop('Rings', axis=1)
y = data['Rings'] - 1

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.LongTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.LongTensor(y_test.values)

In [112]:
class AbaloneClassificationNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(AbaloneClassificationNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [113]:
input_size = X_train.shape[1]
hidden_size = 128
num_classes = 29

net = AbaloneClassificationNet(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes)

In [114]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

In [115]:
num_epochs = 1000
batch_size = 32

for epoch in range(num_epochs):
    net.train()
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        outputs = net(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 2.9190
Epoch [200/1000], Loss: 2.7879
Epoch [300/1000], Loss: 2.7491
Epoch [400/1000], Loss: 2.7333
Epoch [500/1000], Loss: 2.7238
Epoch [600/1000], Loss: 2.7168
Epoch [700/1000], Loss: 2.7110
Epoch [800/1000], Loss: 2.7060
Epoch [900/1000], Loss: 2.7016
Epoch [1000/1000], Loss: 2.6974


In [116]:
net.eval()
with torch.no_grad():
    y_pred = net(X_test_tensor)
    _, predicted = torch.max(y_pred, 1)

y_pred = predicted.numpy() + 1
y_test = y_test + 1

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.1687
